In [1]:
from transformers import pipeline
from langchain_community.document_loaders import PyPDFLoader
import warnings
warnings.filterwarnings('ignore')
import gradio as gr

In [2]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6",device=0)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [9]:
def adaptive_summarize(file):
    load = PyPDFLoader(file.name)
    docs = load.load()

    text_chunks = [doc.page_content for doc in docs if doc.page_content.strip() != ""]
    combined = ""

    for chunk in text_chunks:
        chunk_length = len(chunk.split())
        compression_ratio = 0.5

        max_len = max(50, int(chunk_length * compression_ratio))
        min_len = max(25, int(max_len * 0.5))

        result = summarizer(chunk, max_length=max_len, min_length=min_len, do_sample=False)
        combined += result[0]['summary_text'] + " "

    combined_length = len(combined.split())
    final_compression_ratio = 0.3

    max_len = max(100, int(combined_length * final_compression_ratio))
    min_len = max(50, int(max_len * 0.5))

    final_summary = summarizer(combined, max_length=max_len, min_length=min_len, do_sample=False)

    original_text = " ".join(text_chunks)
    return original_text, final_summary[0]["summary_text"]

In [10]:
with gr.Blocks(title="Adaptive PDF Summarizer") as demo:
    gr.Markdown("## 📄 Adaptive PDF Article Summarizer with GPU")
    with gr.Row():
        up = gr.File(label="Upload a PDF", file_types=[".pdf"])
        btn = gr.Button("Summarize")

    with gr.Row():
        original_output = gr.Markdown(label="Original Text")
        summary_output = gr.Markdown(label="Summarized Text")

    btn.click(fn=adaptive_summarize, inputs=up, outputs=[original_output, summary_output])

In [11]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e8bfe4d5770af92e32.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
